<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

In [1]:
from edc import check_compatibility
check_compatibility("user-2022.02", dependencies=["SH"])



---------

The following environment variables are available:

* `SH_CLIENT_ID`, `SH_INSTANCE_ID`, `SH_CLIENT_NAME`, `SH_CLIENT_SECRET`


![Banner](https://www.sentinel-hub.com/develop/community/contest/resources/img/contest2_image_collection.jpg)

# Custom Script Contest – Urban Growth in Africa
---
Prepared by: _William Ray, Maxim Lamare, and Max Kampen; Sentinel Hub._

**We are pleased to present the next special edition of the Sentinel Hub Custom Script Contest – [Urban Growth in Africa](https://www.sentinel-hub.com/develop/community/contest/#urban-growth-in-africa) – with a focus on observing change detection and growth of urban areas. The Contest started on 3rd May 2021.**

## Getting started with data

In this demonstration Jupyter Notebook, steps to access all the multi-temporal and multi-resolution data available for the contest will be detailed, along with basic processing examples.

## Available datasets

Data from the following satellite products are available for this contest. To jump directly to the section of interest, just click on one of the following products:

- Sentinel-1 [CARD4L](#Sentinel-1-CARD4L)

- Sentinel-1 [GRD](#Sentinel-1-GRD)

- Sentinel-2 [L1C](#Sentinel-2-L1C) & [L2A](#Sentinel-2-L2A)

- Landsat-8 [Collection 2](#Landsat-8-Collection-2) Level 1 & 2

- Airbus [SPOT 6/7](#AIRBUS-SPOT-6/7)

- Airbus [Pléiades](#AIRBUS-Pleiades)

- Planet [PlanetScope](#Planet-PlanetScope)

- Maxar [Worldview & GeoEye](#Maxar-WorldView-/-GeoEye)

## Configuration

Before accessing the data, we will start by importing the necessary Python libraries (already configured in your EDC workspace), and generate credentials automatically to access the services.

In [2]:
# EDC libraries
from edc import setup_environment_variables
from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
from xcube_sh.sentinelhub import SentinelHub
from xcube.core.geom import mask_dataset_by_geometry

# Sentinel Hub
from sentinelhub import BBox, SentinelHubRequest, bbox_to_dimensions, DataCollection, MimeType, SHConfig, geometry

# Utilities
import IPython.display
from os import environ
import matplotlib.pyplot as plt
import datetime as dt

# Numerical computation
import xarray as xr
import numpy as np
import scipy.ndimage as ndi


### BYOC access points

Sentinel-1 CARD4L and the Very High Resolution Commercial collections are provided as pre-processed [BYOC](https://docs.sentinel-hub.com/api/latest/data/byoc/#accessing-data) collections. These collections are accessible via a `collectionID` that will be provided to participants after the registration process.

To be used in this Jupyter Notebook the `collectionID`s need to be set in the `custom.env` file. You can check if the variables are set by executing the next code cell.

If the following env variables are not set, please create a text file named `custom.env` in your home directory with the following input:

```
pleiades_collection = "<collectionIDstring>"
s1grd_collection = "<collectionIDstring>"
spot_collection = "<collectionIDstring>"
planet_collection = "<collectionIDstring>"
maxar_collection = "<collectionIDstring>"
```

In the text file, the `<collectionIDstring>` should be replaced by the `collectionID` of the BYOC dataset that will be provided to you. 

### Credentials generation

The credentials for Sentinel Hub services are automatically set by the Euro Data Cube libraries in the following cell.

In [3]:
# Fetch credentials as environement variables
setup_environment_variables()

# Pass Sentinel Hub credentials to dictionnary
sh_credentials = dict(client_id=environ["SH_CLIENT_ID"],
                      client_secret=environ["SH_CLIENT_SECRET"])

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DAPA_URL`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
# Assign BYOC env variables to variables
pleiades_collection = environ["pleiades_collection"]
s1grd_collection = environ["s1grd_collection"]
spot_collection = environ["spot_collection"]
planet_collection = environ["planet_collection"]
maxar_collection = environ["maxar_collection"]

KeyError: 'pleiades_collection'

## Sentinel-1-CARD4L

Sentinel-1 images are offered as CARD4L datasets, as well as being accessible as standard datasets (see following [section](#Sentinel-1-GRD)).

**CARD4L** stands for "CEOS Analysis Ready Data for Land", and offers satellite data “processed to a minimum set of requirements and organized into a form that allows immediate analysis with a minimum of additional user effort and interoperability both through time and with other datasets” (https://ceos.org/ard/). 

**Mission information**

The Sentinel-1 imagery is provided by two polar-orbiting satellites, operating day and night performing C-band synthetic aperture radar imaging, enabling them to acquire imagery regardless of the weather. More information about the sensor and its characteristics can be found on the [Sentinel Hub documentation page](https://docs.sentinel-hub.com/api/latest/data/sentinel-1-grd/).

The bands available in the Sentinel-1 CARD4L product are the following:

| Name | Description | Resolution |
|-|-|-|
| VH | VV polarisation | 20m |
| VV | VH polarisation | 20m |
| AREA | The normalized scattering area for each output pixel. | 20m |
| MASK | Flags output pixels which are in or near radar shadow. | 20m |
| ANGLE | The local incidence angle for each output pixel. | 20m |

**Data availability**

For practical reasons, the Sentinel-1 CARD4L datasets have been pre-processed and ingested as a [BYOC collection](https://docs.sentinel-hub.com/api/latest/data/byoc/#accessing-data).

Data were ingested for the time-range: 2017-01-01 to 2021-04-15.

### Define area of interest

In the following example, we will set our AOI (covering the Mbeubeuss pond, Dakar, Senegal) using a bounding box expressed as [min_x, min_y, max_x, max_y]. Here the coordinates are in [WGS 84](https://epsg.io/4326) (EPSG **4326**), with units in degrees.

In [ ]:
# Bbox
dakar_large_bbox = [-17.336723, 14.792808, -17.288553, 14.823096]

# Bbox EPSG number
dakar_epsg = 4326

In [ ]:
# Plot the geometry on a map
IPython.display.GeoJSON(BBox(dakar_large_bbox, crs=dakar_epsg).get_geojson())

### Build xcube

In the following cell we will specify the input parameters needed to build the `xcube` array. The following parameters are specified:

- _dataset_name_: the Sentinel Hub identification of the dataset. Here we will call `CUSTOM` because Sentinel-1 CARD4L images are available as a BYOC collection.

- _band_names_: the band names to be used in the xcube array. Here, we will call the VV and Incidence angle bands (`VV` and `ANGLE` respectively).

- _bbox_: the bounding box that sets the extent of the AOI. Because we are using the default WGS84 coordinate system here, the CRS parameter doesn't need to be set.

- _spatial_res_: the spatial resolution of the rasters contained in the xcube array. The spatial resolution is expressed in the units of the coordinate system used. Therefore, in this example, the spatial resolution is set in degrees. For an approximate pixel size of 20 meters, we set the resolution to 0.00018 degrees.

- _time_range_: a list of two dates [start_date, end_date] forming a time period for which all acquisitions will be returned.

- _time_period_: Necessary when requesting a BYOC collection. The Time period string denotes the temporal aggregation period to be used. Here `1d` means that if several images are available for 1 given date, they will be aggregated.

- _collection_id_: the Sentinel Hub BYOC collection ID for Sentinel-1 CARD4L.

In [ ]:
# Setup xcube for S1-CARD4L
cube_config = CubeConfig(dataset_name='CUSTOM',
                         band_names=['VV', 'ANGLE'],
                         bbox=dakar_large_bbox,
                         spatial_res=0.00018,
                         time_range=['2021-01-01', '2021-04-15'],
                         time_period="1d",
                         collection_id=s1grd_collection)

### Open the xcube

In the following cell we open the cube, then filter out the dates that do not contain any data (this is not performed automatically for BYOC datasets in the current version of `xcube`).

_Note:_ Given the large number of acquisitions, this step takes around 1 minute 30.

In [ ]:
%%time
# Open cube
S1_cube = open_cube(cube_config, **sh_credentials)

# Drop empty time slices
S1_cube = S1_cube.dropna(dim="time")

### Explore the xcube

We will first explore the variables and metadata contained in the `xcube` by printing its contents. You can explore the attributes by clicking on the buttons to the right (in the executed version).

In [ ]:
S1_cube

### Plot a timeslice from the xcube

In a second step we will plot the `ANGLE` band (local incidence angle) for a time slice by specifying the desired date. If you are not sure of the exact acquisition date, you can fetch the closest time slice to a given date, as shown below.

_Note_: The format of your date string can be a bit tricky. Here we are matching the format of the dates in the cube for the command to work without having to use `datetime`.

In [ ]:
S1_cube.ANGLE.sel(time='2021-02-01T12:00:00.000000000', method='nearest').plot.imshow(figsize=(12,8), cmap="Greys_r")
plt.show()

### Extract a subset to work on a smaller region

If you would like to work on a subset, e.g. because the region of interest is smaller than the specified bounding box, you can use a WKT polygon or Shapefile to mask the region of interest. Please note however that the WKT / shapefile must be in the same coordinate reference system as the existing `xcube`.

Two methods of selection are possible:

- By masking - this keeps only the data of the WKT Polygon or shapefile

- By clipping - this creates a new bounding box, which ensures that the area of interest is within the subset but keeps the surrounding data.

In this example, we will mask our cube, using the geometry of the Mbeubeuss dump specified in the cell below.

_Note: http://geojson.io/ is a useful tool to extract geometries from a map._

In [ ]:
mbeubeuss_dump_coordinates = {
  "type": "Polygon",
  "coordinates": [[[-17.321186, 14.799613], [-17.321068, 14.79856], [-17.320848, 14.798166], [-17.320107, 14.797798],
                   [-17.319863, 14.797461], [-17.319952, 14.797059], [-17.320483, 14.796498], [-17.320354, 14.795941],
                   [-17.319777, 14.795313], [-17.315692, 14.79981], [-17.314877, 14.799753], [-17.3139, 14.800282], [-17.312661, 14.800168],
                   [-17.312505, 14.800422], [-17.311535, 14.800461], [-17.31044, 14.801169], [-17.310301, 14.802082], [-17.309753, 14.801926],
                   [-17.308949, 14.802144], [-17.308542, 14.802476], [-17.308499, 14.802637], [-17.308402, 14.80273], [-17.308198, 14.802733],
                   [-17.308064, 14.802792], [-17.307911, 14.802743], [-17.307823, 14.80286], [-17.307925, 14.802963], [-17.307833, 14.803306],
                   [-17.307882, 14.803409], [-17.308316, 14.804696], [-17.308112, 14.804779],[-17.307898, 14.804862], [-17.307565, 14.805816],
                   [-17.308038, 14.8065], [-17.308596, 14.806521], [-17.309154, 14.806729], [-17.309905, 14.806231], [-17.310742, 14.806314],
                   [-17.311407, 14.806521], [-17.312051, 14.806272], [-17.312244, 14.805795], [-17.312953, 14.806065], [-17.313339, 14.8065],
                   [-17.314412, 14.806853], [-17.314927, 14.806957], [-17.315291, 14.806832 ], [-17.315656,  14.806169], [-17.31645, 14.805691],
                   [-17.316922, 14.804675], [-17.317394, 14.804654], [-17.318231,14.804322],[-17.31851,14.803077],
                   [-17.318854, 14.802974], [-17.319304, 14.8026], [-17.319314,14.80204],
                   [-17.319057, 14.801833], [-17.319218, 14.801522], [-17.319743, 14.801304], [-17.319914, 14.801122],
                   [-17.320038, 14.800567], [-17.320612, 14.800422], [-17.321186, 14.799613]]]}

In [ ]:
# Mask the cube
Mbeubeuss_dump = mask_dataset_by_geometry(S1_cube, geometry=mbeubeuss_dump_coordinates)

### Plot the masked cube

In the cell below, we will check the output of the previous operation by plotting the VV band for the first time slice `isel(time=0)`.

In [ ]:
# Plot the masked cube
Mbeubeuss_dump.VV.isel(time=0).plot.imshow(cmap='Greys', vmin=0, vmax=1, figsize=(10, 10))
plt.show()

### VV change over time for a given area of interest

In the following cells, the change in VV backscatter will be assessed (for demonstration purposes only) by showing the boxplots of the Mbeubeuss dump for 3 different dates.

The following cell contains a function to convert the long timestamps from the `xcube` to a more readable format.

In [ ]:
def to_datetime(datestring):
    """Convert string to datetime.
    Takes a date in the form of a string and returns a datetime object.
    The string should be in the format YYYY-MM-DD.
    """
    return dt.datetime.strptime(datestring, "%Y-%m-%d")    

### Data preparation

In the following cell, we will prepare the data for plotting in 3 steps:

- we will extract the data from 3 acquisitions that are the closest to 1st January 2021, 1st February 2021 and 1st March 2021.
- we remove the `nan` values from the arrays.
- the timestamps are extracted and converted to a more readable format.

In [ ]:
# Make a list of 3 different dates of interest.
dates_of_interest = [to_datetime("2021-01-01"), to_datetime("2021-01-01"), to_datetime("2021-03-01")]

# Fetch a cube based on the dates
Mbeubeuss_sub_dates = Mbeubeuss_dump.VV.sel(time=dates_of_interest, method="nearest")

# Convert to list for plotting
boxplot_data = [Mbeubeuss_sub_dates.compute().data[0, :, :], Mbeubeuss_sub_dates.compute().data[1, :, :], Mbeubeuss_sub_dates.compute().data[2, :, :]]

# Remove nan values from the data for plotting
boxplot_data = [x[~np.isnan(x)] for x in boxplot_data]

### Data plotting

In the following cell, we will plot the VV backscatter distribution over the Mbeubeuss dump for 3 selected dates as [boxplots](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51).

In [ ]:
# Create figure
fig, ax = plt.subplots(1, figsize=(12,8))

# Plot boxplots
ax.boxplot(boxplot_data)

# Set ticks to times
plt.xticks([1, 2, 3], Mbeubeuss_sub_dates.time.values.astype('datetime64[D]'))

# Plot setup
ax.set_ylabel("VV backscatter")
ax.set_ylim(0, 1)
plt.suptitle("VV backscatter evolution for Mbeubeuss dump, 2021.")
plt.draw()
plt.show()

---

## Sentinel-1 GRD

**Mission information**

The Sentinel-1 imagery is provided by two polar-orbiting satellites, operating day and night performing C-band synthetic aperture radar imaging, enabling them to acquire imagery regardless of the weather. Main applications are for monitoring sea ice, oil spills, marine winds, waves & currents, land-use change, land deformation among others, and to respond to emergencies such as floods and earthquakes.

**S1 GRD**

Sentinel Hub currently supports Sentinel-1 Level-1 GRD (Ground Range Detected) products only. For more information about the sensor's properties and the data available, you can visit the [Sentinel Hub documentation page](https://docs.sentinel-hub.com/api/latest/data/sentinel-1-grd/).

**Data availability**

February 2015 to today.

### Define area of interest

In the following example, we will set our AOI (covering a large part of Dakar, Senegal) using a bounding box expressed as [min_x, min_y, max_x, max_y]. Here the coordinates are in [WGS 84](https://epsg.io/4326) (EPSG **4326**), with units in degrees.

In [ ]:
# Bbox
dakar_bbox = [-17.546573, 14.645375, -17.392724, 14.783846]

# Bbox EPSG
dakar_epsg = 4326

In [ ]:
# Plot the bounding box on a map
IPython.display.GeoJSON(BBox(dakar_bbox,crs=dakar_epsg).get_geojson())

### Check Sentinel-1 GRD available bands

Using EDC inbuilt functions that query Sentinel Hub services, we can easily list the available bands for a given dataset.

In [ ]:
# Create a Sentinel Hub class, using our Sentinel Hub credentials
SH = SentinelHub(**sh_credentials)

# List bands for S1-GRD
SH.band_names('S1GRD')

### Build an xcube

In the following cell we will specify the input parameters needed to build an xcube array. The following parameters are specified:

- _dataset_name_: the Sentinel Hub identification of the dataset. Here we will call `S1GRD` for Sentinel-1 GRD. All available dataset can be listed with `SH.dataset_names`

- _band_names_: the band names to be used in the xcube array (see previous code cell). Here, we will call the VV, HH, VH and HV polarisation bands.

- _bbox_: the bounding box that sets the extent of the AOI. Because we are using the default WGS84 coordinate system here, the CRS parameter doesn't need to be set.

- _spatial_res_: the spatial resolution of the rasters contained in the xcube array. The spatial resolution is expressed in the units of the coordinate system used. Therefore, in this example, the spatial resolution is set in degrees. For an approximate pixel size of 10 meters, we set the resolution to 0.000089 degrees.

- _time_range_: a list of two dates [start_date, end_date] forming a time period for which all acquisitions will be returned. Sentinel-1 GRD data is available from February 2015 onwards. In this example, we will fetch data for all of 2019.

- _time_tolerance_: The tolerance used to identify whether a dataset should still be included within a time period. Here, `30M` corresponds to 30 minutes, thus avoiding duplicate datasets.

_Note: the `return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)` warning can be safely ignored._

In [ ]:
# Setup xcube
cube_config = CubeConfig(dataset_name='S1GRD',
                         band_names=['VV', 'VH'],
                         bbox=dakar_bbox,
                         spatial_res=0.000089,
                         time_range=['2019-01-01', '2019-12-31'],
                         time_tolerance='30M')

### Open the xcube

In the following cell we open the cube and display it's contents. The automatically generated credentials obtained earlier in this Jupyter Notebook are specified as a parameter when opening the cube.

In [ ]:
# Open cube (on the fly)
cube = open_cube(cube_config, **sh_credentials)

# Display contents
cube

### Visualising Urban areas using SAR

We are going to base the visualisation of the Sentinel-1 GRD data on this [custom script](https://custom-scripts.sentinel-hub.com/sentinel-1/urban_areas/) authored by Monja Šebela. Firstly, new variables need to be defined and generated. A specific date can then be called, and then stacked and visualised in a plot. 

Next, to produce the visualisation we need to define and generate Red, Green and Blue channels in the visualisation, calling the VH and VV bands in the cube and adapting them to highlight certain features for visualisation purposes. We do this by defining the new variable and attributing it a "long_name" and "units".

In [ ]:
# Define the Red visualisation
cube['Red'] = 5.5 * cube.VH
cube['Red'].attrs['long_name']='Red'
cube['Red'].attrs['units']='unitless'

# Define the Green visualisation
cube['Green'] = cube.VV
cube['Green'].attrs['long_name']='Green'
cube['Green'].attrs['units']='unitless'

# Define the Blue visualisation
cube['Blue'] = 8 * cube.VH
cube['Blue'].attrs['long_name']='Blue'
cube['Blue'].attrs['units']='unitless'

Secondly, we want to call the new variables that we have named as Red, Green and Blue and stack them into an array that we can then plot. In the example below, we are selecting the time slice closest to the time stamp of "2019-05-23 10:00:00". Lastly, we define the plot and visualise it.

_Note_: The plotting is slow because until you use the data, xcube doesn't perform any computations. A nifty feature to save processing time! As it was the case for the previous data collection, please note that the `time` parameter has to match the format of that found in the xcube.

In [ ]:
# Select the bands and stack them
Green = cube.Green.sel(time='2019-05-23 10:00:00', method='nearest')
Red = cube.Red.sel(time='2019-05-23 10:00:00', method='nearest')
Blue = cube.Blue.sel(time='2019-05-23 10:00:00', method='nearest')

# Stack the three arrays into an RGB image
rgb = np.dstack((Green, Red, Blue))

# Create plot
fig, ax = plt.subplots(1, figsize=(12,8))
ax.imshow(rgb)
ax.set_title(f"Visualisation using False color SAR to map urban areas: {str(cube.time.sel(time='2019-05-23 10:00:00', method='nearest').data).split('T')[0]}")
plt.show()

---

## Sentinel-2 L1C
**Mission information**

SENTINEL-2 is a European wide-swath, high-resolution, multi-spectral imaging mission. Its high-resolution optical images have many applications, including land monitoring, emergency response and security services assistance. The satellite's multispectral imager provides a versatile set of 13 spectral bands spanning from the visible and near infrared to the shortwave infrared.


**Level 1C**

Level 1C offers Top of the atmosphere (TOA) reflectance data. For more information about the sensor's properties and the data available, you can visit the [Sentinel Hub documentation page](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l1c/).

**Data availability**

June 2015 to today.

### Define area of interest

In the following example, we will set our AOI (covering a large part of Dakar, Senegal) using a bounding box expressed as [min_x, min_y, max_x, max_y]. Here the coordinates are in [WGS 84](https://epsg.io/4326) (EPSG **4326**), with units in degrees.

In [ ]:
# Bbox
dakar_bbox = [-17.546573, 14.645375, -17.392724, 14.783846]

# Bbox EPSG
dakar_epsg = 4326

In [ ]:
# Plot the bounding box on a map
IPython.display.GeoJSON(BBox(dakar_bbox,crs=dakar_epsg).get_geojson())

### Check Sentinel-2 L1C available bands

Using EDC inbuilt functions that query Sentinel Hub services, we can list the available bands for a given dataset.

In [ ]:
# Create a Sentinel Hub class
SH = SentinelHub(**sh_credentials)

# List bands for S2-L1C
SH.band_names('S2L1C')

### Build xcube

In the following cell we will specify the input parameters needed to build an xcube array. The following parameters are specified:

- _dataset_name_: the Sentinel Hub identification of the dataset. Here we will call `S2L1C` for Sentinel-2 Level 1C. All available datasets can be listed with `SH.dataset_names`

- _band_names_: the band names to be used in the xcube array (see previous code cell). Here, we will call the Red and Near Infrared bands (B04 and B08 respectively).

- _bbox_: the bounding box that sets the extent of the AOI. Because we are using the default WGS84 coordinate system here, the CRS parameter doesn't need to be set.

- _spatial_res_: the spatial resolution of the rasters contained in the xcube array. The spatial resolution is expressed in the units of the coordinate system used. Therefore, in this example, the spatial resolution is set in degrees. For an approximate pixel size of 10 meters, we set the resolution to 0.000089 degrees.

- _time_range_: a list of two dates [start_date, end_date] forming a time period for which all acquisitions will be returned. Sentinel-2 L1C data is available from November 2015 onwards. In this example, we will fetch data from June to August 2019.

- _time_tolerance_: The tolerance used to identify whether a dataset should still be included within a time period. Here, `30M` corresponds to 30 minutes, thus avoiding duplicate datasets.

_Note: the `return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)` warning can be safely ignored._

In [ ]:
# Setup xcube
cube_config = CubeConfig(dataset_name='S2L1C',
                         band_names=['B04', 'B08'],
                         bbox=dakar_bbox,
                         spatial_res=0.000089,
                         time_range=['2019-06-01', '2019-08-31'],
                         time_tolerance='30M')

### Open the xcube

In the following cell we open the cube and display its contents. The automatically generated credentials obtained earlier in this Jupyter Notebook are specified as a parameter when opening the cube.

In [ ]:
# Open cube (on the fly)
cube = open_cube(cube_config, **sh_credentials)

# Display contents
cube

### Compute an additional band

In the following cell we will compute a new band based on existing bands in the xcube. In this example we will compute the Normalized difference vegetation index (NDVI) for each existing timestamp. [NDVI](https://custom-scripts.sentinel-hub.com/sentinel-2/ndvi/) is a commonly used index to quantify green vegetation, and is derived from Sentinel-2 bands 4 and 8.

Firstly, we define the NDVI formula:

In [ ]:
# Define NDVI equation
cube['NDVI'] = (cube.B08 - cube.B04) / (cube.B04 + cube.B08)

# Set band attributes
cube['NDVI'].attrs['long_name'] = 'Normalized Difference Vegetation Index'
cube['NDVI'].attrs['units'] = 'unitless'

In [ ]:
# Display the calculated NDVI band
cube.NDVI

Now we have generated this new band, let's visualise the values for a pixel over the time period. In this example, the coordinates of a point located in an Urban reserve (Réserve Naturelle Urbaine de la Grande Niaye de Pikine et Dépendance) were selected. 

_Note:_ Because the function goes through all the time slices, it may take some time to compute the data.

In [ ]:
%%time
# Set latitude and longitude of point to query
point_lat = 14.7678
point_lon = -17.4023

# Get timeseries
timeseriesNDVI = cube.NDVI.sel(lat=point_lat, lon=point_lon, method='nearest').to_series()

### Plot NDVI timeseries for a given location
Using matplotlib we can visualise how NDVI changes over time.

In [ ]:
# Plot NDVI timeseries
fig, ax = plt.subplots(1, figsize=(15, 6))
ax.plot(timeseriesNDVI, color="green", marker="o", linewidth=0.5)
ax.set_title(f"NDVI timeseries. Latitude: {point_lat}, Longitude: {point_lon}.")
plt.show()

---

## Sentinel-2 L2A
**Mission information**

SENTINEL-2 is a European wide-swath, high-resolution, multi-spectral imaging mission. Its high-resolution optical images have many applications, including land monitoring, emergency response and security services assistance. The satellite's multispectral imager provides a versatile set of 13 spectral bands spanning from the visible and near infrared to the shortwave infrared.


**Level 2A**

Level 2A offers Bottom of the atmosphere (BOA) reflectance data. For more information about the sensor's properties and the data available, you can visit the [Sentinel Hub documentation page](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/).

**Data availability**

January 2017 to today.

### Define area of interest

In the following example, we will set our AOI (covering a large part of Dakar, Senegal) using a bounding box expressed as [min_x, min_y, max_x, max_y]. Here the coordinates are in [WGS 84](https://epsg.io/4326) (EPSG **4326**), with units in degrees.

In [ ]:
# Bbox
dakar_bbox = [-17.546573, 14.645375, -17.392724, 14.783846]

# Bbox EPSG
dakar_epsg = 4326

In [ ]:
# Plot the bounding box on a map
IPython.display.GeoJSON(BBox(dakar_bbox,crs=dakar_epsg).get_geojson())

### Check Sentinel-2 L2A available bands

Using EDC inbuilt functions that query Sentinel Hub services, we can list the available bands for a given dataset.

In [ ]:
# Create a Sentinel Hub class
SH = SentinelHub(**sh_credentials)

# List bands for S2-L2A
SH.band_names('S2L2A')

### Build xcube

In the following cell we will specify the input parameters needed to build an xcube array. The following parameters are specified:

- _dataset_name_: the Sentinel Hub identification of the dataset. Here we will call `S2L2A` for Sentinel-2 Level 2A. All available dataset can be listed with `SH.dataset_names`

- _band_names_: the band names to be used in the xcube array (see previous code cell). Here, we will call the Red and Near Infrared bands (B04 and B08 respectively).

- _bbox_: the bounding box that sets the extent of the AOI. Because we are using the default WGS84 coordinate system here, the CRS parameter doesn't need to be set. For an example with a different coordinate system, see the section for [Airbus SPOT 6/7](#AIRBUS-SPOT-6/7).

- _spatial_res_: the spatial resolution of the rasters contained in the xcube array. The spatial resolution is expressed in the units of the coordinate system used. Therefore, in this example, the spatial resolution is set in degrees. For an approximate pixel size of 10 meters, we set the resolution to 0.000089 degrees.

- _time_range_: a list of two dates [start_date, end_date] forming a time period for which all acquisitions will be returned. Sentinel-2 L2A data is available from January 2017 onwards. In this example, we will fetch data for May 2019.

- _time_tolerance_: The tolerance used to identify whether a dataset should still be included within a time period. Here, `30M` corresponds to 30 minutes, thus avoiding duplicate datasets.

_Note: the `return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)` warning can be safely ignored._

In [ ]:
# Setup xcube
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04', 'B08'],
                         bbox=dakar_bbox,
                         spatial_res=0.000089,
                         time_range=['2019-05-01', '2019-05-31'],
                         time_tolerance='30M')

### Open the xcube

In the following cell we open the cube and display its contents. The automatically generated credentials obtained earlier in this Jupyter Notebook are specified as a parameter when opening the cube.

In [ ]:
# Open cube (on the fly)
cube = open_cube(cube_config, **sh_credentials)

# Display contents
cube

### Compute and visualise an additional band

In the following cell we will compute a new band based on existing bands in the xcube. In this example we will compute the Normalized difference vegetation index (NDVI) for each existing timestamp. [NDVI](https://custom-scripts.sentinel-hub.com/sentinel-2/ndvi/) is a commonly used index to quantify green vegetation, and is derived from Sentinel-2 bands 4 and 8.

Firstly, we define the NDVI formula and create a new variable attributing a "long_name" and "units".

In [ ]:
# Define NDVI equation and set as band
cube['NDVI'] = (cube.B08 - cube.B04) / (cube.B04 + cube.B08)
cube['NDVI'].attrs['long_name'] = 'Normalized Difference Vegetation Index'
cube['NDVI'].attrs['units'] = 'unitless'

Secondly, we will call this variable for a specific date and then visualise it. In this example, we will specify a date of interest and fetch the closest acquisition in time available. We also filter out values smaller than 0, using the `where` function.

In [ ]:
# Plot NDVI in shades of Green where NDVI is greater than 0. 
cube.NDVI.sel(time='2019-05-23 10:00:00', method='nearest').where(cube.NDVI.sel(time='2019-05-23 10:00:00', method='nearest') > 0).plot.imshow(vmin=0, vmax=1, cmap='Greens', figsize=(18, 15))
plt.show()

---

## Landsat-8 Collection 2

**Mission information**

Landsat 8 is the most recently launched Landsat satellite (provided by NASA/USGS) and carries the Operational Land Imager (OLI) and the Thermal Infrared Sensor (TIRS) instruments. These two sensors provide seasonal coverage of the global landmass. Landsat-8 Level-1 provides global top-of-atmosphere reflectance and brightness-temperature, whereas Level-2 provides global surface reflectance and surface temperature science products. Landsat Level-2 science products are generated from Collection 2 Level-1 inputs that meet the <76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product.

**Collection 2**

For more information about the sensor's properties and the data available, you can visit the Sentinel Hub documentation pages for [Level 1](https://docs.sentinel-hub.com/api/latest/data/landsat-8/) and [Level2](https://docs.sentinel-hub.com/api/latest/data/landsat-8-l2/). Additional information about Collection 2 can be found on the [dedicated USGS page](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2-level-2-science-products).

**Data availability**

Since February 2013.


### Fetch an image from Landsat-8 Collection 2 Level 1 and Level 2

Below we define the cells to retrieve data for Landsat 8 Collection 2. 

First, we need to set the input parameters. Currently, Collection 2 data is not available through the `xcube` package. Therefore, in this example, we will access the data from Sentinel Hub using the [sentinelhub-py library](https://sentinelhub-py.readthedocs.io/en/latest/index.html).

In [ ]:
# Sentinel Hub configuration for the Python package
config = SHConfig()
config.sh_client_id = environ["SH_CLIENT_ID"]
config.sh_client_secret = environ["SH_CLIENT_SECRET"]

The Collection 2 is only recently available through Sentinel Hub services and the `sentinelhub-py` package has not yet been updated to contain the settings for the Collection 2 by default. This means that the `DataCollection` object has to be specified manually, as shown below.

In [ ]:
# Define a new DataCollection for Level-1 data
DataCollection.define_from(
            DataCollection.LANDSAT8,
            "LOTL1",
            service_url="https://services-uswest2.sentinel-hub.com",
            wfs_id="DSS12",  # https://www.sentinel-hub.com/develop/api/ogc/standard-parameters/wfs/
            api_id="LOTL1",
            catalog_id="landsat-8-l1c",
            processing_level="L1C",
            bands=("B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09", "B10", "B11", "BQA", "QA_RADSAT", "VAA", "VZA", "SAA", "SZA", "dataMask"),
)

In [ ]:
# Define a new DataCollection for Level-2 data
DataCollection.define_from(
            DataCollection.LANDSAT8,
            "LOTL2",
            service_url="https://services-uswest2.sentinel-hub.com",
            wfs_id="DSS13",  # https://www.sentinel-hub.com/develop/api/ogc/standard-parameters/wfs/
            api_id="LOTL2",
            catalog_id="landsat-8-l2a",
            processing_level="L2A",
            bands=("B01", "B02", "B03", "B04", "B05", "B06", "B07", "B10", "BQA", "QA_RADSAT", "SR_QA_AEROSOL", "ST_QA", 
                   "ST_TRAD", "ST_URAD", "ST_DRAD", "ST_ATRAN", "ST_EMIS", "ST_EMSD", "ST_CDIST", "dataMask"),
)

### Define area of interest

In the following example, we will set our AOI (covering a large part of Dakar, Senegal) using a bounding box expressed as [min_x, min_y, max_x, max_y]. Here the coordinates are in [WGS 84](https://epsg.io/4326) (EPSG **4326**), with units in degrees. We also calculate the AOI in pixels based upon the resolution of Landsat 8 which is 30m.

_Note: with `sentinelhub-py` , the size of the image cannot be more than 2500px * 2500px, else the request will fail._

In [ ]:
dakar_bbox = [-17.546573, 14.645375, -17.392724, 14.783846]
dakar_epsg = 4326

resolution = 30
dakar_bbox = BBox(bbox=dakar_bbox, crs=dakar_epsg)
dakar_size = bbox_to_dimensions(dakar_bbox, resolution=resolution)

print(f'Image shape at {resolution} m resolution: {dakar_size} pixels')

### Define an evalscript

Now that the input parameters are defined we will use an [evalscript](https://docs.sentinel-hub.com/api/latest/evalscript/) to visualise the data that we are requesting. In this example, we are requesting Band 3, 4 & 5 (Green, Red, NIR) and returning them in a 3-band image. 

In [ ]:
evalscript_false_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B05", "B04", "B03"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B05, sample.B04, sample.B03];
    }
"""

### Create a Sentinel Hub request for Level 1 and Level 2 data

Based on the parameters set in the previous cells, we will build a Sentinel Hub request for Landsat-8 Collection 2 Level 1 and Level 2, that will return False Colour RGB png images. For more information on using `sentinelhub-py`, you can refer to the [documentation](https://sentinelhub-py.readthedocs.io/en/latest/).

In [ ]:
l8_l1c = SentinelHubRequest(
    evalscript=evalscript_false_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.LOTL1,
            time_interval=('2020-05-01', '2020-08-30'),
            other_args = {"dataFilter":{"mosaickingOrder":"leastCC"}}
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.PNG)
    ],
    bbox=dakar_bbox,
    size=dakar_size,
    config=config
)

l8_l2a = SentinelHubRequest(
    evalscript=evalscript_false_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.LOTL2,
            time_interval=('2020-05-01', '2020-08-30'),
            other_args = {"dataFilter":{"mosaickingOrder":"leastCC"}}
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.PNG)
    ],
    bbox=dakar_bbox,
    size=dakar_size,
    config=config
)

### Run the requests

In [ ]:
false_colour_l1c = l8_l1c.get_data()
false_colour_l2a = l8_l2a.get_data()

### Plot the false colour RGB images

In the next cell, we will use a plotting function for the request results (https://github.com/sentinel-hub/sentinelhub-py/blob/master/examples/utils.py).

In [ ]:
def plot_image(image, factor=1.0, clip_range = None, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 12))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()

Lastly, let's visualise the Level 1 and 2 images! The image type is UINT8 which means that the image bands have values of 0-255. To visualise them on a scale of 0 to 1 we need to divide by 255 first. You can also adjust the factor to brighten the image if you wish. 

In [ ]:
# Level 1
image_l1c = false_colour_l1c[0]

# plot function
# factor 1/255 to scale between 0-1
# factor 2.5 to increase brightness
plot_image(image_l1c, factor=2.5/255, clip_range=(0,1))

In [ ]:
# Level 2
image_l2a = false_colour_l2a[0]

# plot function
# factor 1/255 to scale between 0-1
# factor 2.5 to increase brightness
plot_image(image_l2a, factor=2.5/255, clip_range=(0,1))

---

## AIRBUS SPOT 6/7

![Airbus](https://docs.sentinel-hub.com/api/latest/static/e5660ba66c215f7378a8843ac68b5743/logo.png)

SPOT 6/7 is a satellite constellation providing very high-resolution optical imagery and is owned by [Airbus](https://www.intelligence-airbusds.com/en/8693-spot-67).

**Mission information**

SPOT 6/7 is composed of two twin satellites orbiting the Earth 180° apart. The following table lists the bands available through Sentinel Hub services and their resolution:

| Name | Description | Resolution |
|-|-|-|
| B0 | Blue, 454-519 nm | 6m |
| B1 | Green, 527-587 nm | 6m |
| B2 | Red, 624-694 nm | 6m |
| B3 | Near Infrared, 756-880 nm | 6m |
| PAN | Panchromatic, 455-744 nm | 1.5m |
| dataMask | The mask of data/no data pixels | N/A |


**Data availability**

SPOT 6/7 data can be accessed as a [BYOC collection](https://docs.sentinel-hub.com/api/latest/data/byoc/#accessing-data), i.e. by specifying a `collection_ID`.

Images acquired on the following dates are available within the collection:

- 2012-10-17
- 2012-10-27
- 2012-12-06
- 2012-12-08
- 2012-12-15
- 2012-12-18
- 2012-12-20
- 2012-12-27
- 2013-03-13
- 2014-10-19
- 2015-02-21
- 2015-04-20
- 2015-12-23
- 2016-04-24
- 2017-03-22
- 2017-04-17
- 2018-03-01
- 2018-10-22
- 2019-10-20
- 2020-10-18
- 2021-02-25

### Define area of interest

In the following example, we will set our AOI (covering the port of Dakar) using a GeoJSON polygon geometry. Here the coordinates are in [WGS 84 / Pseudo-Mercator](https://epsg.io/3857) (EPSG **3857**), with units in meters.

In [ ]:
# Bbox
dakar_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        -1939522.159759,
        1654269.238697
      ],
      [
        -1939187.64469,
        1653887.045024
      ],
      [
        -1939331.024194,
        1653724.663382
      ],
      [
        -1939206.791642,
        1653437.99502
      ],
      [
        -1939025.229553,
        1652922.092265
      ],
      [
        -1939178.405172,
        1651173.533333
      ],
      [
        -1940544.740602,
        1651555.800865
      ],
      [
        -1940994.026067,
        1652434.969169
      ],
      [
        -1941127.720775,
        1652883.886191
      ],
      [
        -1940410.934574,
        1654020.886953
      ],
      [
        -1939503.012807,
        1654718.303547
      ],
      [
        -1939455.256746,
        1654555.916536
      ],
      [
        -1939522.159759,
        1654269.238697
      ]
    ]
  ]
}

# Bbox EPSG number and code (see https://docs.sentinel-hub.com/api/latest/api/process/crs/)
dakar_epsg= 3857
dakar_projection = "http://www.opengis.net/def/crs/EPSG/0/3857"

# Build a Sentinel Hub geometry object from the input polygon
dakar_geom = geometry.Geometry(dakar_geometry,crs=dakar_epsg)

In [ ]:
# Plot the geometry on a map
IPython.display.GeoJSON(dakar_geom.transform(4326).get_geojson())

### Query two images using Sentinel Hub services

In addition to `xcube`, you can also use Sentinel Hub services to query images and analyse them. This approach is recommended if you want to compare a small amount of images spread over a large period of time.

In the following example, we will retrieve RGB images for 2 dates, on the 17th October 2012 and on 25th February 2021 over Dakar's industrial harbour.

#### Input parameters

To call a BYOC collection, we first need to setup the input parameters for the request:

- we start by defining the `DataCollection` based on the `collectionID`

- we set the [Evalscript](https://docs.sentinel-hub.com/api/latest/evalscript/), which tells Sentinel Hub services how to process the data and what product to return: here we will return the R,G,B product (B2, B1, B0 bands) as UINT16 images and a fourth band (`dataMask`) representing the nodata values (outside our area of interest) and plotted as an alpha band.

- we build a SentinelHubRequest for each acquisition, that also include:

    - The date that interest us in the `time_interval` parameter
    - The output file format (not important here as we will not save the data)
    - The geometry set previously
    - The resolution (in the units of the CRS)
    - A config file that contains credentials (reused from the Landsat example, so make sure the cells above are executed first).

In [ ]:
# Define DataCollection
spot_col = DataCollection.define_byoc(spot_collection)

# Evalscript to return RGB True Color
tc_eval = """
//VERSION=3
function setup() {
  return {
    input: ["B0", "B1", "B2", "dataMask"],
    output: { bands: 4, sampleType: "UINT16" }, 
  };
}

function evaluatePixel(sample) {
  return [sample.B2, sample.B1, sample.B0, sample.dataMask * 65535];
}
"""

# Build request for the first time step
request_t1 = SentinelHubRequest(
    evalscript=tc_eval,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=spot_col,
            time_interval=('2012-10-17')
        )],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    geometry=dakar_geom,
    resolution=(6, 6),
    config=config
    )

# Build a request for the second time step
request_t2 = SentinelHubRequest(
    evalscript=tc_eval,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=spot_col,
            time_interval=('2021-02-25')
        )],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    geometry=dakar_geom,
    resolution=(6, 6),
    config=config
    )

In [ ]:
# Run the requests
t1_rgb = request_t1.get_data()[0]
t2_rgb = request_t2.get_data()[0]

## Basic processing example: NDWI change

In the following cells, the change in Normalized Difference Water Index ([NDWI](https://custom-scripts.sentinel-hub.com/sentinel-2/ndwi/)) will be assessed (for demonstration purposes only).

### Plot RGB images

In a first step we will plot the True Color RGB of Dakar harbor for each date side-by-side. 

To visualise RGB images, we need to normalise the bands between 0 and 1. SPOT data are expressed in `DN` as `DN = reflectance * 10000`. The typical range for a SPOT image is 0 - 4000. Therefore we will normalise 0 - 4000 to 0 - 1 for visualisation purposes.

In [ ]:
# Plot RGB images together
fig, (ax, ax2) = plt.subplots(1,2, figsize=(12,8))

# Set ticks to times
ax.imshow(t1_rgb / 4000)
ax2.imshow(t2_rgb / 4000)

# Set dates as titles
ax.set_title("Date: 2012-10-17")
ax2.set_title("Date: 2021-02-25")

plt.show()

### Plot change in NDWI

To monitor change in NDWI, and potentially detect areas encroached on the ocean, we will plot the difference in NDWI between the two available dates. Looking at the RGB images and the NDWI change map (large red zone), we can see a new area in the port that was built out over the ocean.

In [ ]:
# Evalscript to return NDWI
ndwi_eval = """
//VERSION=3
function setup() {
  return {
    input: ["B1", "B3"],
    output: { bands: 1, sampleType: "FLOAT32" }, 
  };
}

function evaluatePixel(sample) {
  let ndwi = (sample.B1 - sample.B3) / (sample.B1 + sample.B3)
  return [ndwi];
}
"""

# Build request for the first time step
request_t1_ndwi = SentinelHubRequest(
    evalscript=ndwi_eval,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=spot_col,
            time_interval=('2012-10-17')
        )],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    geometry=dakar_geom,
    resolution=(6, 6),
    config=config
    )

# Build a request for the second time step
request_t2_ndwi = SentinelHubRequest(
    evalscript=ndwi_eval,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=spot_col,
            time_interval=('2021-02-25')
        )],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    geometry=dakar_geom,
    resolution=(6, 6),
    config=config
    )

In [ ]:
# Calculate NDWI
ndwi_t1 = request_t1_ndwi.get_data()[0]
ndwi_t2 = request_t2_ndwi.get_data()[0]

# Compute change between two dates
ndwi_change = ndwi_t2 - ndwi_t1

In [ ]:
# Plot NDWI change
fig, ax = plt.subplots(1, figsize=(12,8))

# Set ticks to times
change = ax.imshow(ndwi_change[:,:], cmap="RdBu",) #vmin=-0.5, vmax=0.5)
plt.colorbar(change)
ax.set_title("NDWI change between 2012 and 2021")

plt.show()

---

## AIRBUS Pleiades

![Airbus](https://docs.sentinel-hub.com/api/latest/static/e5660ba66c215f7378a8843ac68b5743/logo.png)

Pléiades is a satellite constellation providing very high-resolution optical imagery and is owned by [Airbus](https://www.intelligence-airbusds.com/en/8693-spot-67).

**Mission information**

Pléiades is composed of two twin satellites orbiting the Earth 180° apart. The satellites deliver 0.5 m optical imagery and offer a daily revisit capability to any point on the globe. The following table lists the bands available through Sentinel Hub services and their resolution:

| Name | Description | Resolution |
|-|-|-|
| B0 | Blue, 430-550 nm | 2m |
| B1 | Green, 490-610 nm | 2m |
| B2 | Red, 600-720 nm | 2m |
| B3 | Near Infrared, 750-950 nm | 2m |
| PAN | Panchromatic, 480-830 nm | 0.5m |
| dataMask | The mask of data/no data pixels | N/A |


**Data availability**

Pléiades data can be accessed as a [BYOC collection](https://docs.sentinel-hub.com/api/latest/data/byoc/#accessing-data), i.e. by specifying a `collection_ID`.

Images acquired on the following dates are available within the collection:

- 2012-03-13
- 2012-05-06
- 2012-10-02
- 2013-02-23
- 2013-10-20
- 2014-10-14
- 2015-11-07
- 2016-03-10
- 2016-12-08
- 2017-10-09
- 2017-10-15
- 2019-03-06
- 2019-12-24
- 2020-06-24
- 2020-10-18

More acquisitions will be added during the contest, stay tuned for news!

### Define area of interest

In the following example, we will set our AOI (covering the Mbao industrial area, Dakar, Senegal - see http://www.sar.sn/) using a GeoJSON polygon geometry. Here the coordinates are in [WGS 84 / UTM zone 28N](https://epsg.io/32628) (EPSG **32628**), with units in meters.

In [ ]:
# Bbox
mbao_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        246910.013036,
        1630461.326004
      ],
      [
        246752.787159,
        1631786.114943
      ],
      [
        248484.369728,
        1631997.760134
      ],
      [
        248751.899486,
        1631930.66922
      ],
      [
        248656.087821,
        1631614.745376
      ],
      [
        248804.78545,
        1630340.550845
      ],
      [
        248371.768609,
        1630501.232138
      ],
      [
        248252.122139,
        1630121.242429
      ],
      [
        246910.013036,
        1630461.326004
      ]
    ]
  ]
}

# Bbox EPSG number and code (see https://docs.sentinel-hub.com/api/latest/api/process/crs/)
mbao_epsg = 32628
mbao_projection= "http://www.opengis.net/def/crs/EPSG/0/32628"

# Build a Sentinel Hub geometry object from the input polygon
mbao_geom = geometry.Geometry(mbao_geometry,crs=mbao_epsg)

In [ ]:
# Plot the geometry on a map
IPython.display.GeoJSON(mbao_geom.transform(4326).get_geojson())

### Build xcube

In the following cell we will specify the input parameters needed to build an xcube array. The following parameters are specified:

- _dataset_name_: the Sentinel Hub identification of the dataset. Here we will call `CUSTOM` because Pléiades images are available as a BYOC collection.

- _band_names_: the band names to be used in the xcube array. Here, we will call the Blue, Green, Red and Panchromatic bands (B0, B1, B2, PAN respectively).

- _bbox_: the bounding box that sets the extent of the AOI. The bounding box covering the input geometry can be directly called from the `sentinelhub.geometry` object created above.

- _crs_: if the coordinate reference system isn't in WGS84 (EPSG:4326) as the default value, it is specified using an [opengis URL](https://docs.sentinel-hub.com/api/latest/api/process/crs/). Here, we are using EPSG:32628.

- _spatial_res_: the spatial resolution of the rasters contained in the xcube array. The spatial resolution is expressed in the units of the coordinate system used. Therefore, in this example, the spatial resolution is set in meters. Since we are also requesting the panchromatic band, and will perform pan-sharpening, we will request the cube at 0.5 meters resolution.

- _time_range_: a list of two dates [start_date, end_date] forming a time period for which all acquisitions will be returned. For this example, we will request a single date.

- _time_period_: Necessary when requesting a BYOC collection. The time period string denotes the temporal aggregation period to be used. Here `1d` means that if several images are available for 1 given date (not the case), they will be aggregated.

- _collection_id_: the Sentinel Hub BYOC collection ID for Airbus Pléiades.

In [ ]:
# Setup xcube for Pléiades
cube_config = CubeConfig(dataset_name='CUSTOM',
                         band_names=['B0', 'B1', 'B2', 'PAN'],
                         bbox=mbao_geom.bbox,
                         crs=mbao_projection,
                         spatial_res=0.5,
                         time_range=['2019-12-24', '2019-12-24'],
                         time_period="1d",
                         collection_id=pleiades_collection)

### Open cube

In [ ]:
Pleiades_cube = open_cube(cube_config, **sh_credentials)

### Explore the `xcube`

We will first explore the variables and metadata contained in the `xcube` by printing its contents.

In [ ]:
Pleiades_cube

### Plot a timeslice from the xcube

In a second step we will plot Band PAN (panchromatic) for a time slice by selecting it's number (0...n; 0 being the first slice and n the last). Here we only have 1 date in our cube, so we will plot `time=0`. Please note that in this step the cube is being requested on the fly for the first time, therefore this step will be a bit slow.

In [ ]:
%%time
Pleiades_cube.PAN.isel(time=0).plot.imshow(cmap='Greys_r', vmin=0, vmax=4000, figsize=(10, 10))
plt.show()

## Extract a subset to work on a smaller region

If you would like to work on a subset, e.g. because the region of interest is smaller than the specified bounding box, you can use a WKT polygon or Shapefile to mask the region of interest. Please note however that the WKT / shapefile must be in the same coordinate reference system as the existing `xcube`.

Two methods of selection are possible:

- By masking - this keeps only the data of the WKT Polygon or shapefile

- By clipping - this creates a new bounding box, which ensures that the area of interest is within the subset but keeps the surrounding data.

In this example, we will mask our cube, using the geometry specified earlier.

In [ ]:
# Mask the cube
Mbao_cube = mask_dataset_by_geometry(Pleiades_cube, geometry=mbao_geom.get_geojson())

In [ ]:
# Plot the masked cube
Mbao_cube.PAN.isel(time=0).plot.imshow(cmap='Greys_r', vmin=0, vmax=4000, figsize=(12, 10))
plt.show()

## Basic processing example: create a pan-sharpened RGB visualisation

The Red, Green and Blue bands of Pléiades have a resolution of 2m, whereas the panchromatic band has a resolution of 0.5m. By combining the bands, using a method called [pan-sharpening](https://www.geosage.com/highview/imagefusion.html), we can visualise a "sharpened" RGB True Color image at a resolution of 0.5 meters. 

Several methods can be used to pan-sharpen a Pléiades RGB image. In this example, we will use the Brovey algorithm (eq.2-3 in this [paper](http://jurnal.lapan.go.id/index.php/ijreses/article/view/2609)).

### Calculate pansharpened R, G, and B bands

In a first step we will calculate pan-sharpened bands for the Red, Green and Blue channels, using the panchromatic band as input. The bands will be stored in the cube for later re-use if needed. Note that the bands are not computed until use, meaning that this step is very fast.

In [ ]:
Mbao_cube["PAN_RED"] = (Mbao_cube.B2 / (Mbao_cube.B0 + Mbao_cube.B1 + Mbao_cube.B2)) * Mbao_cube.PAN;
Mbao_cube["PAN_GREEN"] = (Mbao_cube.B1 / (Mbao_cube.B0 + Mbao_cube.B1 + Mbao_cube.B2)) * Mbao_cube.PAN
Mbao_cube["PAN_BLUE"] = (Mbao_cube.B0 / (Mbao_cube.B0 + Mbao_cube.B1 + Mbao_cube.B2)) * Mbao_cube.PAN

### Plot pan-sharpened RGB true color of Pléiades

In the following cell, the bands calculated above are used for the first time, and therefore only computed in this step. The first execution of this cell will be thus be slower than the previous steps.

In [ ]:
# Plot RGB images together
fig, ax = plt.subplots(1, figsize=(12,10))

# Build rgb combination and plot
rgb_tc = np.dstack((Mbao_cube.PAN_RED.isel(time=0), Mbao_cube.PAN_GREEN.isel(time=0), Mbao_cube.PAN_BLUE.isel(time=0)))
ax.imshow(rgb_tc / 2000)

# Set date as title
ax.set_title(f"Date: {str(Mbao_cube.time[0].data).split('T')[0]}")

plt.show()

---

## Planet PlanetScope

![Planet logo](https://docs.sentinel-hub.com/api/latest/static/fd3d797a803db45fe5dd5a4d1490315a/logo.png)

The PlanetScope satellite constellation consists of more than 130 small satellites called Doves. Each Dove satellite is a CubeSat made of three cubic units and thus measures only 10 cm x 10 cm x 30 cm. 

**Mission information**

PlanetScope satellite deliver optical imagery with an almost daily coverage worldwide at a spatial resolution of 3.7-4.1 m. The following table lists the bands available through Sentinel Hub services and their resolution:

| Name | Description | Resolution |
|-|-|-|
| B1 | Blue, 455 - 515 nm | 3m (resampled) |
| B2 | Green, 500 - 590 nm | 3m (resampled) |
| B3 | Red, 590 - 670 nm | 3m (resampled) |
| B4 | Near Infrared, 780 - 860 nm | 3m (resampled) |
| UDM | Unusable Data Mask (see [docs](https://assets.planet.com/docs/Planet_Combined_Imagery_Product_Specs_letter_screen.pdf#page=91)) | 3m (resampled) |
| dataMask | The mask of data/no data pixels | N/A |


**Data availability**

Pléiades data can be accessed as a [BYOC collection](https://docs.sentinel-hub.com/api/latest/data/byoc/#accessing-data), i.e. by specifying a `collection_ID`.

The images available for this contest were obtained under the “Hectares under management” model, meaning that you will have access to all images between **1st January 2017 and mid-May 2021** covering the eastern part of Dakar (of interest for Urban expansion).

### Define area of interest

In the following example, we will set our AOI (north of Rufisque) using a bounding box expressed as [min_x, min_y, max_x, max_y]. Here the coordinates are in [UTM Zone 28 N](https://epsg.io/326328) (EPSG **32628**), with units in meters.

In [ ]:
# Bbox
dakar_large_bbox = [249984, 1636430, 251910, 1638457]

# Bbox EPSG number
dakar_epsg = 32628

In [ ]:
# Plot the geometry on a map
IPython.display.GeoJSON(BBox(dakar_large_bbox, crs=dakar_epsg).transform(4326).get_geojson())

### Build xcube

In the following cell we will specify the input parameters needed to build the `xcube` array. The following parameters are specified:

- _dataset_name_: the Sentinel Hub identification of the dataset. Here we will call `CUSTOM` because PlanetScope images are available as a BYOC collection.

- _band_names_: the band names to be used in the xcube array. Here, we will call the 4 bands: Blue, Green, Red, Near Infrared (`B1`, `B2`, `B3`, and `B4` respectively).

- _bbox_: the bounding box that sets the extent of the AOI. Because we are using a UTM coordinate system here, the CRS parameter will have to be set.

- _crs_: if the coordinate reference system isn't in WGS84 (EPSG:4326) as the default value, it is specified using an [opengis URL](https://docs.sentinel-hub.com/api/latest/api/process/crs/). Here, we are using EPSG:32628. 

- _spatial_res_: the spatial resolution of the rasters contained in the xcube array. The spatial resolution is expressed in the units of the coordinate system used. Therefore, in this example, the spatial resolution is set in meters.

- _time_range_: a list of two dates [start_date, end_date] forming a time period for which all acquisitions will be returned.

- _time_period_: Necessary when requesting a BYOC collection. The Time period string denotes the temporal aggregation period to be used. Here `1d` means that if several images are available for 1 given date, they will be aggregated.

- _collection_id_: the Sentinel Hub BYOC collection ID for PlanetScope.

In [ ]:
# Setup xcube for S1-CARD4L
cube_config = CubeConfig(dataset_name='CUSTOM',
                         band_names=['B1', 'B2', 'B3', 'B4'],
                         bbox=dakar_large_bbox,
                         crs="http://www.opengis.net/def/crs/EPSG/0/32628",
                         spatial_res=3,
                         time_range=['2017-04-21', '2017-04-24'],
                         time_period="1d",
                         collection_id=planet_collection)

### Open the xcube

In the following cell we open the cube and explore its contents.

In [ ]:
# Open cube
Planet_cube = open_cube(cube_config, **sh_credentials)

# Drop empty time slices
Planet_cube

### Plot two acquisitions within the xcube

In the following cell we will compare the acquisitions of two dates contained within the xcube, that show rubbish-burning events occuring in the Mbeubeuss dump. To plot the False Colour composite, we will stack the NIR, Red and Green bands (divided by 10000 to obtain reflectance and multiplied by a factor of 3 for better visualisation).

In [ ]:
# Define factor for viewing the reflectance
f = 10000 / 3

# Stack bands for each acquisition
rgb_t1 = np.dstack((Planet_cube.B4.isel(time=0) / f, Planet_cube.B3.isel(time=0)  / f, Planet_cube.B2.isel(time=0) / f))
rgb_t2 = np.dstack((Planet_cube.B4.isel(time=3) / f, Planet_cube.B3.isel(time=3)  / f, Planet_cube.B2.isel(time=3) / f))

# Create plot
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(10, 10))
ax.imshow(rgb_t1)
ax2.imshow(rgb_t2)

# Plot options
ax.set_xticks([])
ax.set_yticks([])
ax.set_title(Planet_cube.time.isel(time=0).values.astype('datetime64[D]'))
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_title(Planet_cube.time.isel(time=3).values.astype('datetime64[D]'))
fig.suptitle("Planetscope False Colour Composite", y=0.75)
plt.show()

### Compute a mask and overlay on the RGB False Colour Composite

For demonstration purposes, the following cells will show how to perform morphological operations and apply masks on an image. A simple approach will be used to identify the smoke plume in the images: since smoke scatters light strongly in the blue wavelengths of the light spectrum, we will first attempt to identify the smoke plume by applying an empirical threshold to Band 1.


In [ ]:
# Set threshold
thresh = 1800

# Create a mask by first setting 0 where the threshold insn't met, 
# then setting 1 where the threshold is met
Planet_cube["bright"] = Planet_cube.B1.where(Planet_cube.B1 > thresh, 0)
Planet_cube["bright"] = Planet_cube["bright"].where(Planet_cube["bright"] == 0 , 1)

In [ ]:
Planet_cube["bright"].isel(time=0).plot.imshow()
plt.show()

The simple thresholding seems to perform quite well on the first image, as shown in the plot above. However a number of smaller false positive patches appear. To remove them, we can apply an [erosion-dilation](https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html) step (also called binary opening) to the image using the `scipy` libraries.

In [ ]:
# Generate a structure for the filter
struct = np.ones((5,5))

# Perform cleaning step on each time step
clean_t1 = ndi.morphology.binary_opening(Planet_cube["bright"].isel(time=0), struct)
clean_t2 = ndi.morphology.binary_opening(Planet_cube["bright"].isel(time=3), struct)

In [ ]:
# Plot the masks
fig, (ax, ax2) = plt.subplots(1,2, figsize=(10, 10))
ax.imshow(clean_t1)
ax2.imshow(clean_t2)
plt.show()

## Plot masked image

In the last step we plot the masked False Colour composite, using the cleaned smoke layer.

In [ ]:
fig, (ax, ax2) = plt.subplots(1,2, figsize=(10, 10))

ax.imshow(rgb_t1)
ax.imshow(np.ma.masked_where(clean_t1!=1, clean_t1), cmap="YlGn")
ax2.imshow(rgb_t2)
ax2.imshow(np.ma.masked_where(clean_t2!=1, clean_t2), cmap="YlGn")
plt.show()

### Conclusions 

The approach shown here is too simplistic to be used for any analysis. Indeed, the darker (grey) parts of the smoke plume are not detected, and some bright surfaces are misinterpreted as smoke. However, large parts of the smoke plume are detected, and the method could be refined to perform in a more robust manner.

---

## Maxar WorldView / GeoEye

![Maxar](https://docs.sentinel-hub.com/api/latest/static/f078cb7581036c9400e466b13a65e296/logo.png)

The WorldView constellations consists of four active satellites: WorldView-1 (data not available in Sentinel Hub), GeoEye-1 (GE01), WorldView-2 (WV02), and WorldView-3 (WV03). The WorldView-4 (WV04) satellite was operational from November 2016 to January 2019 and the data it acquired is available in Sentinel Hub.

**Mission information**

Worldview satellites deliver optical imagery with a revisit time of 1 day to 3 days depending on the satellite. Note that the data is in general not acquired systematically. Archive data is available sporadically over an area of interest. The resolution of images available in Sentinel Hub is 0.5 m for the panchromatic band and 2 m for multispectral bands. The following table lists the bands available through Sentinel Hub services and their resolution:

| Name | Description | Resolution |
|-|-|-|
| Blue | Blue, 450 - 510 nm | 2m |
| Green | Green, 510 - 580 nm | 2m |
| Red | Red, 630 - 690 nm for WV02 and WV03, 655 - 690 nm for GE01 and WV04 | 2m |
| NearIR1 | Near Infrared, 770 - 895 nm for WV02 and WV03, 780 - 920 nm for GE01 and WV04 | 2m |
| PAN | Panchromatic, 450 - 800 nm | 0.5m |
| dataMask | The mask of data/no data pixels | N/A |


**Data availability**

Maxar data can be accessed as a [BYOC collection](https://docs.sentinel-hub.com/api/latest/data/byoc/#accessing-data), i.e. by specifying a `collection_ID`.

Data is available for the following dates / date ranges:

- 2013-02-26

- 2015-03-04 

- 2017-04-15 / 2017-04-25

- 2019-03-01 / 2019-03-03

- 2021-04-09 / 2021-04-23

### Define area of interest

In the following example, we will set our AOI (Keur Ndiaye Lo, north of Rufisque) using a bounding box expressed as [min_x, min_y, max_x, max_y]. Here the coordinates are in [UTM Zone 28 N](https://epsg.io/326328) (EPSG **32628**), with units in meters.

In [ ]:
# Bbox
Keur_Ndiaye = [259051, 1631160, 259563, 1631603]

# Bbox EPSG number
dakar_epsg = 32628

In [ ]:
# Plot the geometry on a map
IPython.display.GeoJSON(BBox(Keur_Ndiaye, crs=dakar_epsg).transform(4326).get_geojson())

### Build xcube

In the following cell we will specify the input parameters needed to build the `xcube` array. The following parameters are specified:

- _dataset_name_: the Sentinel Hub identification of the dataset. Here we will call `CUSTOM` because Maxar images are available as a BYOC collection.

- _band_names_: the band names to be used in the xcube array. Here, we will call the panchromatic band (`PAN`).

- _bbox_: the bounding box that sets the extent of the AOI. Because we are using a UTM coordinate system here, the CRS parameter will have to be set.

- _crs_: if the coordinate reference system isn't in WGS84 (EPSG:4326) as the default value, it is specified using an [opengis URL](https://docs.sentinel-hub.com/api/latest/api/process/crs/). Here, we are using EPSG:32628. 

- _spatial_res_: the spatial resolution of the rasters contained in the xcube array. The spatial resolution is expressed in the units of the coordinate system used. Therefore, in this example, the spatial resolution is set in meters.

- _time_range_: a list of two dates [start_date, end_date] forming a time period for which all acquisitions will be returned.

- _time_period_: Necessary when requesting a BYOC collection. The Time period string denotes the temporal aggregation period to be used. Here `1d` means that if several images are available for 1 given date, they will be aggregated.

- _collection_id_: the Sentinel Hub BYOC collection ID for Maxar.

In [ ]:
# Setup xcube for S1-CARD4L
cube_config = CubeConfig(dataset_name='CUSTOM',
                         band_names=['PAN'],
                         bbox=Keur_Ndiaye,
                         crs="http://www.opengis.net/def/crs/EPSG/0/32628",
                         spatial_res=0.5,
                         time_range=['2013-02-26', '2021-04-09'],
                         time_period="1d",
                         collection_id=maxar_collection)

### Open the xcube

In the following cell we open the cube and explore its contents.

In [ ]:
# Open cube
maxar_cube = open_cube(cube_config, **sh_credentials)

# Drop empty time slices
maxar_cube

## Filter the cube

If we look at the cube in the cell above, we can see 2964 time slices (many of which are empty). Since we know the dates that we would like to keep in our xcube, we will create a subset based on the list of dates and overwrite our cube on the fly.

In [ ]:
# List of dates of interest
dates_of_interest = ["2013-02-26", "2015-03-04", "2017-04-15", "2019-03-01", "2021-04-09"]

# Subset
maxar_cube = maxar_cube.where(maxar_cube.time.dt.date.isin(np.array(dates_of_interest, dtype='datetime64')), drop=True)

In [ ]:
# Check the cube again
maxar_cube

## Plot the panchromatic band for each time stamp

Now we have our subset determined, we will make an RGB image for each timestamp and plot the urban expansion over the years using the automatic plotting funtion over time.

In [ ]:
# Stack bands for each acquisition
pan_plot = maxar_cube["PAN"].plot.imshow(figsize=(12,8), x="x", y="y", col="time", col_wrap=3, cmap="Greys_r", vmin=500, vmax=4000)
plt.show()

---

### Now it's your turn...

With the examples provided above, you should have all the elements needed to access the data and start coding away. We look forward to your results!